**Table of contents**<a id='toc0_'></a>    
- [About](#toc1_)    
- [Data importation](#toc2_)    
- [Data conversion](#toc3_)    
- [Exploratory Data Analysis](#toc4_)    
- [Drop columns](#toc5_)    
- [Remove features](#toc6_)    
  - [Features with too many nulls](#toc6_1_)    
  - [Features with near zero variance](#toc6_2_)    
- [Create features](#toc7_)    
  - [Scale data](#toc7_1_)    
  - [Create time features](#toc7_2_)    
  - [Check if monotonic](#toc7_3_)    
  - [Add seasons](#toc7_4_)    
- [Train test split](#toc8_)    
- [One Hot Encoding](#toc9_)    
- [Train & evaluate model](#toc10_)    
  - [Quantitative evaluation](#toc10_1_)    
  - [Qualitative evaluation](#toc10_2_)    
- [Make predictions](#toc11_)    
- [Export data](#toc12_)    
- [Charts & Graphs](#toc13_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [73]:
import polars as pl
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.metrics import make_scorer, root_mean_squared_error
import pandas as pd
import numpy as np
from datetime import datetime
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, HoltWinters
from sklearn.pipeline import Pipeline

# <a id='toc1_'></a>[About](#toc0_)
This notebook illustrates how to perform time-series modelling using the Craigslist vehicle sales as sample data.

In [2]:
pd.options.display.max_columns = None
N_ROWS = 1000

In [3]:
def display_pl(df: pl.LazyFrame):
    if N_ROWS is not None:
        display(df.head().collect())

# <a id='toc2_'></a>[Data importation](#toc0_)
Import all data without inferring to make data importation faster. Columns will be converted to different data types on demand.

In [4]:
craigslist_vehicles = pl.scan_csv("./data/craigslist_vehicles.csv", n_rows=N_ROWS, infer_schema_length=0)
display_pl(craigslist_vehicles)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date,removal_date
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""362773""","""7307679724""","""https://abilen…","""abilene""","""https://abilen…","""4500""","""2002.0""","""bmw""","""x5""",null,null,"""gas""","""184000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$4,500 Cash 2…",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-16 00:…","""2021-05-02 00:…"
"""362712""","""7311833696""","""https://abilen…","""abilene""","""https://abilen…","""4500""","""2002.0""","""bmw""","""x5""",null,null,"""gas""","""184000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$4,500 Cash 2…",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-24 00:…","""2021-04-28 00:…"
"""362722""","""7311441996""","""https://abilen…","""abilene""","""https://abilen…","""4900""","""2006.0""","""toyota""","""camry""","""excellent""","""4 cylinders""","""gas""","""184930.0""","""clean""","""automatic""",null,"""fwd""",null,"""sedan""","""silver""","""https://images…","""2006 TOYOTA CA…",null,"""tx""","""32.453848""","""-99.7879""","""2021-04-23 00:…","""2021-05-25 00:…"
"""362771""","""7307680715""","""https://abilen…","""abilene""","""https://abilen…","""6500""","""2008.0""","""ford""","""expedition""",null,null,"""gas""","""206000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$6500.00 2008 …",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-16 00:…","""2021-04-26 00:…"
"""362710""","""7311834578""","""https://abilen…","""abilene""","""https://abilen…","""6500""","""2008.0""","""ford""","""expedition""",null,null,"""gas""","""206000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$6500.00 2008 …",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-24 00:…","""2021-05-12 00:…"


In [5]:
# Confirming the date format
if N_ROWS is not None:
    craigslist_vehicles.collect().sample(20).select(pl.col("removal_date")).to_series().to_list()

# <a id='toc3_'></a>[Data conversion](#toc0_)
Converting specific columns to different data types.

In [6]:
numeric_cols = ["price", "odometer"]
date_cols = ["posting_date", "removal_date"]

In [7]:
def convert_data(data: pl.LazyFrame, date_columns: list = date_cols, numeric_columns: list = numeric_cols) -> pl.LazyFrame:
    
    for d in date_columns:
        data = data.with_columns(pl.col(d).str.to_datetime(format="%Y-%m-%d %H:%M:%S%z"))
        
    for n in numeric_columns:
        data = data.with_columns(pl.col(n).cast(pl.Float32()))
    
    data = data.sort(by="removal_date", descending=False)
    data = data.drop_nulls(subset="removal_date")
    return data

craigslist_vehicles = convert_data(craigslist_vehicles)
display_pl(craigslist_vehicles)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date,removal_date
str,str,str,str,str,f32,str,str,str,str,str,str,f32,str,str,str,str,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]"
"""362868""","""7301632975""","""https://abilen…","""abilene""","""https://abilen…",21995.0,"""2017.0""","""ford""","""mustang""",null,null,"""gas""",33532.0,"""clean""","""automatic""","""1FA6P8TH2H5277…","""rwd""",null,"""coupe""","""silver""","""https://images…","""Stanley Ford S…",null,"""tx""","""32.447591""","""-100.408954""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""362854""","""7302149036""","""https://abilen…","""abilene""","""https://abilen…",32995.0,"""2016.0""","""ram""","""1500 laramie 4…",null,null,"""diesel""",83116.0,"""clean""","""automatic""",null,null,null,null,null,"""https://images…","""CALL OR TEXT 5…",null,"""tx""","""30.663""","""-97.9316""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""362855""","""7302141697""","""https://abilen…","""abilene""","""https://abilen…",21950.0,"""2014.0""","""gmc""","""sierra 1500""","""excellent""","""8 cylinders""","""gas""",143000.0,"""clean""","""automatic""",null,"""4wd""",null,null,null,"""https://images…","""2014 GMC Sierr…",null,"""tx""","""32.392""","""-99.7746""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""288440""","""7301733200""","""https://akronc…","""akron / canton…","""https://akronc…",31990.0,"""2017.0""","""audi""","""a4 allroad pre…","""good""",null,"""other""",22297.0,"""clean""","""other""","""WA17NAF44HA169…",null,null,"""wagon""","""black""","""https://images…","""Carvana is the…",null,"""oh""","""41.08""","""-81.52""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""288372""","""7302301841""","""https://akronc…","""akron / canton…","""https://akronc…",26995.0,"""2008.0""","""ram""","""2500""",null,null,"""gas""",130960.0,"""clean""","""automatic""","""3D7KS28A18G194…",null,null,"""other""","""red""","""https://images…","""2008 Dodge Ram…",null,"""oh""","""40.85882""","""-81.57895""",2021-04-05 00:00:00 UTC,2021-04-06 00:00:00 UTC


In [8]:
min_sale_date = craigslist_vehicles.select("removal_date").min().collect().to_series()[0]

# <a id='toc4_'></a>[Exploratory Data Analysis](#toc0_)
Perform high level Exploratory Data Analysis (EDA).

In [9]:
(craigslist_vehicles
 .collect()
 .describe()
 .transpose(column_names="describe", include_header=True)
)

column,count,null_count,mean,std,min,25%,50%,75%,max
str,str,str,str,str,str,str,str,str,str
"""""","""1000""","""0""",null,null,"""286278""",null,null,null,"""362870"""
"""id""","""1000""","""0""",null,null,"""7301595551""",null,null,null,"""7317043421"""
"""url""","""1000""","""0""",null,null,"""https://abilen…",null,null,null,"""https://akronc…"
"""region""","""1000""","""0""",null,null,"""abilene""",null,null,null,"""akron / canton…"
"""region_url""","""1000""","""0""",null,null,"""https://abilen…",null,null,null,"""https://akronc…"
"""price""","""1000.0""","""0.0""","""23814.68554687…","""68227.5234375""","""0.0""","""6499.0""","""16999.0""","""29995.0""","""1234567.0"""
"""year""","""998""","""2""",null,null,"""1950.0""",null,null,null,"""2021.0"""
"""manufacturer""","""981""","""19""",null,null,"""acura""",null,null,null,"""volvo"""
"""model""","""996""","""4""",null,null,"""1 series 128i …",null,null,null,"""yukon xl"""


# <a id='toc5_'></a>[Drop columns](#toc0_)
Drops columns that cannot be used as features. These are columns such as identifiers, urls, text descriptions, etc.

In [10]:
id_cols = ['', "id", "url", "region_url", "VIN", "image_url", "description", "lat", "long", "year"]
craigslist_vehicles = craigslist_vehicles.drop(id_cols)

# <a id='toc6_'></a>[Remove features](#toc0_)
Removes additional columns that cannot be used as features. These are columns such as those that have no variance, with too many null values, or have too many unique values that cannot be consolidated into fewer values.

## <a id='toc6_1_'></a>[Features with too many nulls](#toc0_)
Identify columns that have greater than a certain threshold of null values.

In [11]:
def find_excess_nulls(data: pl.LazyFrame, thr: float = 0.2) -> list:
    df = (data
        .null_count()
        .collect()
        .transpose(include_header=True, column_names=["null_count"])
        .with_columns(pl.lit(value=len(craigslist_vehicles.collect())).alias("obs"))
        .with_columns((pl.col("null_count") / pl.col("obs")).alias("prop"))
        .with_columns((pl.col("prop") > thr).alias("is_excess_nulls"))
        .filter(pl.col("is_excess_nulls") == True)
    )
    
    print(df)
        
    excess_nulls = (df
        .select("column")
        .to_series().to_list()
    )
    
    return excess_nulls

excess_null_cols = find_excess_nulls(craigslist_vehicles)
excess_null_cols

shape: (6, 5)
┌─────────────┬────────────┬──────┬───────┬─────────────────┐
│ column      ┆ null_count ┆ obs  ┆ prop  ┆ is_excess_nulls │
│ ---         ┆ ---        ┆ ---  ┆ ---   ┆ ---             │
│ str         ┆ u32        ┆ i32  ┆ f64   ┆ bool            │
╞═════════════╪════════════╪══════╪═══════╪═════════════════╡
│ condition   ┆ 588        ┆ 1000 ┆ 0.588 ┆ true            │
│ cylinders   ┆ 706        ┆ 1000 ┆ 0.706 ┆ true            │
│ drive       ┆ 424        ┆ 1000 ┆ 0.424 ┆ true            │
│ size        ┆ 862        ┆ 1000 ┆ 0.862 ┆ true            │
│ paint_color ┆ 328        ┆ 1000 ┆ 0.328 ┆ true            │
│ county      ┆ 1000       ┆ 1000 ┆ 1.0   ┆ true            │
└─────────────┴────────────┴──────┴───────┴─────────────────┘


['condition', 'cylinders', 'drive', 'size', 'paint_color', 'county']

## <a id='toc6_2_'></a>[Features with near zero variance](#toc0_)
Identifies columns with little to no variation in their values.

In [12]:
def find_nzv_categorical(data: pl.LazyFrame, thr: float = 0.8) -> list:
    """Identifies categorical columns with a dominant feature that's greater than a certain threshold."""
    
    cols = data.select(pl.col(pl.Utf8)).columns
    
    categorical_cols = [c 
        for c in cols 
        if c not in 
        numeric_cols + excess_null_cols + id_cols
        ]
    
    df = (data
    .select(categorical_cols)
    .melt(variable_name="column")
    .group_by(pl.all())
    .len()
    .rename({"len": "null_count"})
    .with_columns(pl.col("null_count").sum().over("column").alias("total"))
    .with_columns((pl.col("null_count") / pl.col("total")).alias("prop"))
    .with_columns((pl.col("prop") > thr).alias("is_nzv"))
    .sort(by="column")
    .filter(pl.col("is_nzv") == True)
    .collect()
    )
    
    print(df)
    
    is_nzv = (df
    .select("column")
    .to_series().to_list()
    )
    
    return is_nzv

is_nzv_categorical = find_nzv_categorical(craigslist_vehicles)
is_nzv_categorical

shape: (1, 6)
┌──────────────┬───────┬────────────┬───────┬───────┬────────┐
│ column       ┆ value ┆ null_count ┆ total ┆ prop  ┆ is_nzv │
│ ---          ┆ ---   ┆ ---        ┆ ---   ┆ ---   ┆ ---    │
│ str          ┆ str   ┆ u32        ┆ u32   ┆ f64   ┆ bool   │
╞══════════════╪═══════╪════════════╪═══════╪═══════╪════════╡
│ title_status ┆ clean ┆ 992        ┆ 1000  ┆ 0.992 ┆ true   │
└──────────────┴───────┴────────────┴───────┴───────┴────────┘


['title_status']

In [13]:
def find_nzv_numeric(data: pl.LazyFrame, num_cols: list = numeric_cols, thr: float = 0.8) -> list:
    """Identifies numeric columns with little to no variance in their values."""
    
    numeric_data = data.select(num_cols).with_columns(pl.all().cast(pl.Float32()))
    nzv = VarianceThreshold(thr * (1 - thr))
    nzv.fit_transform(numeric_data.collect())
    idx = nzv.get_support(indices=False)
    retained_feats = nzv.get_feature_names_out()[idx]
    return [f for f in numeric_data.columns if f not in retained_feats]

is_nzv_numeric = find_nzv_numeric(craigslist_vehicles)
is_nzv_numeric

[]

In [14]:
# Consolidate all the columns to remove
cols_to_drop = set(id_cols + excess_null_cols + is_nzv_categorical + is_nzv_numeric)
craigslist_vehicles = craigslist_vehicles.drop(cols_to_drop)

display_pl(craigslist_vehicles)

region,price,manufacturer,model,fuel,odometer,transmission,type,state,posting_date,removal_date
str,f32,str,str,str,f32,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]"
"""abilene""",21995.0,"""ford""","""mustang""","""gas""",33532.0,"""automatic""","""coupe""","""tx""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""abilene""",32995.0,"""ram""","""1500 laramie 4…","""diesel""",83116.0,"""automatic""",null,"""tx""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""abilene""",21950.0,"""gmc""","""sierra 1500""","""gas""",143000.0,"""automatic""",null,"""tx""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""akron / canton…",31990.0,"""audi""","""a4 allroad pre…","""other""",22297.0,"""other""","""wagon""","""oh""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""akron / canton…",26995.0,"""ram""","""2500""","""gas""",130960.0,"""automatic""","""other""","""oh""",2021-04-05 00:00:00 UTC,2021-04-06 00:00:00 UTC


# <a id='toc7_'></a>[Create features](#toc0_)
Create specific features of interest.

## <a id='toc7_1_'></a>[Scale data](#toc0_)
Add price and odomoter bands. A better approach here would be to scale the values.

In [15]:
def create_scaled_features(data: pl.LazyFrame, scaler = StandardScaler()) -> pl.LazyFrame:
    """Scale numeric features and return the original dataframe with the scaled features."""
    
    numeric_feat = data.select(pl.col(pl.NUMERIC_DTYPES))
    cols = numeric_feat.columns
    categorical_feat = data.drop(cols)
    print("Numeric features:", cols)
    
    scaled_data = scaler.fit_transform(numeric_feat.collect())
    pd_scaled = pd.DataFrame(scaled_data, columns=cols)
    pl_scaled = pl.LazyFrame(pd_scaled)
    
    return pl.concat(items=[categorical_feat, pl_scaled], how="horizontal")

craigslist_vehicles_scaled = create_scaled_features(craigslist_vehicles)
display_pl(craigslist_vehicles_scaled)

Numeric features: ['price', 'odometer']


region,manufacturer,model,fuel,transmission,type,state,posting_date,removal_date,price,odometer
str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64
"""abilene""","""ford""","""mustang""","""gas""","""automatic""","""coupe""","""tx""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC,0.134622,-0.250896
"""abilene""","""ram""","""1500 laramie 4…","""diesel""","""automatic""",null,"""tx""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC,-0.026684,-1.000003
"""abilene""","""gmc""","""sierra 1500""","""gas""","""automatic""",null,"""tx""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC,-0.027344,0.653822
"""akron / canton…","""audi""","""a4 allroad pre…","""other""","""other""","""wagon""","""oh""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC,0.119884,-1.16974
"""akron / canton…","""ram""","""2500""","""gas""","""automatic""","""other""","""oh""",2021-04-05 00:00:00 UTC,2021-04-06 00:00:00 UTC,0.230012,0.598542


## <a id='toc7_2_'></a>[Create time features](#toc0_)
Determine how long it has taken to sell a vehicle in terms of years, months, and days.

In [16]:
group_by_cols = craigslist_vehicles_scaled.columns + ["year_sold", "month_sold", "day_sold"]
group_by_cols = [c for c in group_by_cols if not c.endswith("date")]

def create_time_features(data: pl.LazyFrame, time_col: str = "removal_date", group_by: list = group_by_cols) -> pl.LazyFrame:
    """Extract the year, month, and day from a specific time column."""
    
    res = (data
        .with_columns(
            pl.col(time_col).dt.year().alias("cal_year"),
            pl.col(time_col).dt.month().alias("month_sold"),
            pl.col(time_col).dt.day().alias("day_sold")
        )
        .with_columns((pl.col("cal_year").min() - pl.col("cal_year")).alias("year_sold"))
        .group_by(pl.col(group_by), maintain_order=True)
        .len()
        .rename({"len": "count"})
    )
    
    return res

model_data = create_time_features(craigslist_vehicles_scaled)
display_pl(model_data)

region,manufacturer,model,fuel,transmission,type,state,price,odometer,year_sold,month_sold,day_sold,count
str,str,str,str,str,str,str,f64,f64,i32,i8,i8,u32
"""abilene""","""ram""","""1500 laramie 4…","""diesel""","""automatic""",null,"""tx""",0.134622,-0.250896,0,4,5,1
"""abilene""","""ford""","""mustang""","""gas""","""automatic""","""coupe""","""tx""",-0.026684,-1.000003,0,4,5,1
"""abilene""","""gmc""","""sierra 1500""","""gas""","""automatic""",null,"""tx""",-0.027344,0.653822,0,4,5,1
"""akron / canton…","""audi""","""a4 allroad pre…","""other""","""other""","""wagon""","""oh""",0.119884,-1.16974,0,4,5,1
"""abilene""","""ford""","""f-250""","""gas""","""automatic""","""truck""","""tx""",0.230012,0.598542,0,4,6,1


## <a id='toc7_3_'></a>[Check if monotonic](#toc0_)
Determine if the data is monotonically increasing.

In [17]:
def create_removal_date(data: pd.DataFrame | pl.LazyFrame) -> pd.DataFrame:
    if isinstance(data, pl.LazyFrame):
        data = data.collect().to_pandas()
    
    year_sold = data.year_sold.astype("Int8")
    
    removal_year = [c + min_sale_date.year for c in year_sold]
    removal_month = data.month_sold.astype("Int8")
    removal_day = data.day_sold.astype("Int8")
    
    removal_date = []
    for y, m, d in zip(removal_year, removal_month, removal_day):
        try:
            r = datetime(y, m, d)
            removal_date.append(r)
        except TypeError:
            removal_date.append(None)
    
    return data.assign(removal_date = removal_date)

In [18]:
assert create_removal_date(model_data).removal_date.is_monotonic_increasing, "The dataset should increase monotonically"

## <a id='toc7_4_'></a>[Add seasons](#toc0_)
Change month of the year into season. This may not be necessary to the model but would be useful for visualisation.

In [19]:
def map_month_to_season(month):
    seasons = {
        "spring": [3, 4, 5],
        "summer": [6, 7, 8],
        "autumn": [9, 10, 11],
        "winter": [12, 1, 2]
    }

    for season, months in seasons.items():
        if month in months:
            return season
        else:
            return "unknown"

def create_seaons(data: pl.LazyFrame) -> pl.LazyFrame:
    res = data.with_columns(
        pl.col("month_sold").map_elements(function=map_month_to_season, skip_nulls=False).alias("season")
    )
    return res

model_data = create_seaons(model_data)
display_pl(model_data)

region,manufacturer,model,fuel,transmission,type,state,price,odometer,year_sold,month_sold,day_sold,count,season
str,str,str,str,str,str,str,f64,f64,i32,i8,i8,u32,str
"""abilene""","""ram""","""1500 laramie 4…","""diesel""","""automatic""",null,"""tx""",0.134622,-0.250896,0,4,5,1,"""spring"""
"""abilene""","""ford""","""mustang""","""gas""","""automatic""","""coupe""","""tx""",-0.026684,-1.000003,0,4,5,1,"""spring"""
"""abilene""","""gmc""","""sierra 1500""","""gas""","""automatic""",null,"""tx""",-0.027344,0.653822,0,4,5,1,"""spring"""
"""akron / canton…","""audi""","""a4 allroad pre…","""other""","""other""","""wagon""","""oh""",0.119884,-1.16974,0,4,5,1,"""spring"""
"""abilene""","""ford""","""f-250""","""gas""","""automatic""","""truck""","""tx""",0.230012,0.598542,0,4,6,1,"""spring"""


# <a id='toc8_'></a>[Train test split](#toc0_)
Split the data into train/test. Being time series data, we will use scikit-learn's `TimeSeriesSplit`.

In [20]:
y_data = model_data.select("count").collect().to_series()
y = pd.Series(y_data)

X = model_data.drop("count").collect().to_pandas()
print(X.shape)

(998, 13)


In [21]:
# We want the forecase for next week
ts_cv = TimeSeriesSplit(n_splits=5, gap=7)
splits = list(ts_cv.split(X, y))

In [22]:
for i, (tr, te) in enumerate(splits):
    tr_len, te_len = len(tr), len(te)
    print(f"split {i} has {tr_len} training indices and {te_len} testing indices: test prop > {te_len/tr_len}")

split 0 has 161 training indices and 166 testing indices: test prop > 1.031055900621118
split 1 has 327 training indices and 166 testing indices: test prop > 0.5076452599388379
split 2 has 493 training indices and 166 testing indices: test prop > 0.3367139959432049
split 3 has 659 training indices and 166 testing indices: test prop > 0.251896813353566
split 4 has 825 training indices and 166 testing indices: test prop > 0.2012121212121212


In [23]:
# split with test proportion of 30%
SPLIT_30_PERC = 2

train_idx, test_idx = splits[SPLIT_30_PERC]
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [24]:
# The max date of an obs in train should be less than the min date of an obs in test
print("Max removal date from train")
max_removal_date_train = create_removal_date(X_train).removal_date.max()
print(max_removal_date_train)

print("\nMin removal date from test")
min_removal_date_test = create_removal_date(X_test).removal_date.min()
print(min_removal_date_test)

diff_days = (min_removal_date_test - max_removal_date_train).days

assert create_removal_date(X).removal_date.is_monotonic_increasing, "Data should increase monotonically"

Max removal date from train
2021-05-05 00:00:00

Min removal date from test
2021-05-06 00:00:00


# <a id='toc9_'></a>[One Hot Encoding](#toc0_)
Perform One Hot Encoding of the dataset to convert categorical features to numeric.

In [25]:
ohe = OneHotEncoder(drop="if_binary", max_categories=5, dtype=np.int8)
ohe.fit(X)

OneHotEncoder(drop='if_binary', dtype=<class 'numpy.int8'>, max_categories=5)

In [26]:
def ohe_dataframe(data: pd.DataFrame, encoder: OneHotEncoder = ohe) -> pd.DataFrame:
    """One hot encodes the dataframe and labels the columns of the resulting dataframe"""
    
    X = encoder.transform(data)
    X_df = pd.DataFrame(X.toarray(), columns=ohe.get_feature_names_out())
    print("The infrequent categories are:", len(encoder.infrequent_categories_))
    return X_df

X_train_encoded = ohe_dataframe(X_train)
display(X_train_encoded)

The infrequent categories are: 13


,region_akron / canton,manufacturer_chevrolet,manufacturer_ford,manufacturer_jeep,manufacturer_toyota,manufacturer_infrequent_sklearn,model_2500,model_f-150,model_grand caravan,model_impala,model_infrequent_sklearn,fuel_diesel,fuel_gas,fuel_hybrid,fuel_other,fuel_infrequent_sklearn,transmission_automatic,transmission_manual,transmission_other,type_SUV,type_pickup,type_sedan,type_None,type_infrequent_sklearn,state_tx,price_-0.2905806826421867,price_-0.2759165078746724,price_-0.2619855418455338,price_-0.26125233310715806,price_infrequent_sklearn,odometer_-1.5065991230126392,odometer_1.03913180275432,odometer_1.1655088572695873,odometer_1.7267347773094142,odometer_infrequent_sklearn,year_sold_0,month_sold_4,month_sold_5,month_sold_6,day_sold_6,day_sold_10,day_sold_15,day_sold_19,day_sold_infrequent_sklearn,season_unknown
0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0
489,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0
490,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0
491,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0


# <a id='toc10_'></a>[Train & evaluate model](#toc0_)
## <a id='toc10_1_'></a>[Quantitative evaluation](#toc0_)
Trains and evaluates the resulting ML model using the `Root Mean Square Error (RMSE)` evaluation metric.

In [86]:
def train_model(X_train, X_test, y_train, y_test, cv = ts_cv, regressor = HistGradientBoostingRegressor()):
    model = regressor.fit(X_train, y_train)
    
    X_test_encoded = ohe_dataframe(data=X_test)
    y_pred = model.predict(X_test_encoded)
    rmse = root_mean_squared_error(y_true=y_test, y_pred=y_pred)
    
    print("TESTING ERROR:")
    print("rmse:", rmse)
    print("\n")
    
    X_encoded = ohe_dataframe(X)
    scorer = make_scorer(score_func=root_mean_squared_error)
    cv_scores = cross_val_score(estimator=model, X=X_encoded, y=y, scoring=scorer, cv=cv)
    
    print("TESTING ERRORS:")
    print("cv_scores:", cv_scores)
    print("rmse:", cv_scores.mean())
    print("std:", cv_scores.std())

    return model

hgbr_model = train_model(X_train=X_train_encoded, X_test=X_test, y_train=y_train, y_test=y_test)

The infrequent categories are: 13
TESTING ERROR:
rmse: 0.07770566326225252


The infrequent categories are: 13
TESTING ERRORS:
cv_scores: [0.01797554 0.00768578 0.07770566 0.01774947 0.0118536 ]
rmse: 0.026594010558160536
std: 0.025844558069478493


In [85]:
def train_using_pipeline(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series, cv: TimeSeriesSplit = ts_cv):
    pipe = Pipeline(steps=[
        ("ohe", OneHotEncoder(drop="if_binary", max_categories=5, dtype=np.int8, handle_unknown="infrequent_if_exist")),
        ("to_dense", FunctionTransformer(lambda x: x.toarray() if hasattr(x, "toarray") else x, accept_sparse=True)),
        ("regressor", HistGradientBoostingRegressor())
    ])
        
    model = pipe.fit(X_train, y_train)
    # X_test_encoded = ohe_dataframe(data=X_test)
    y_pred = model.predict(X_test)
    rmse = root_mean_squared_error(y_true=y_test, y_pred=y_pred)
    
    print("TESTING ERROR:")
    print("rmse:", rmse)
    print("\n")
    
    scorer = make_scorer(score_func=root_mean_squared_error)
    X_encoded = ohe_dataframe(X)
    cv_scores = cross_val_score(estimator=model, X=X_encoded, y=y, scoring=scorer, cv=cv)
    
    print("TESTING ERRORS:")
    print("cv_scores:", cv_scores)
    print("rmse:", cv_scores.mean())
    print("std:", cv_scores.std())
        
    return model
    
hgbr_model_2 = train_using_pipeline(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

/Users/johnmwangi/opt/miniconda3/envs/interviews/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [1, 2, 7, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


TESTING ERROR:
rmse: 0.07769535823855618


The infrequent categories are: 13


/Users/johnmwangi/opt/miniconda3/envs/interviews/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [13, 32] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/johnmwangi/opt/miniconda3/envs/interviews/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [27, 36, 37] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


TESTING ERRORS:
cv_scores: [0.01797554 0.00768578 0.07770566 0.01774947 0.0118536 ]
rmse: 0.026594010558160536
std: 0.025844558069478493


/Users/johnmwangi/opt/miniconda3/envs/interviews/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [38, 44] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [87]:
print(hgbr_model_2)

Pipeline(steps=[('ohe',
                 OneHotEncoder(drop='if_binary', dtype=<class 'numpy.int8'>,
                               handle_unknown='infrequent_if_exist',
                               max_categories=5)),
                ('to_dense',
                 FunctionTransformer(accept_sparse=True,
                                     func=<function train_using_pipeline.<locals>.<lambda> at 0x292967c70>)),
                ('regressor', HistGradientBoostingRegressor())])


In [28]:
craigslist_vehicles.collect().head(20)

region,price,manufacturer,model,fuel,odometer,transmission,type,state,posting_date,removal_date
str,f32,str,str,str,f32,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]"
"""abilene""",32995.0,"""ram""","""1500 laramie 4…","""diesel""",83116.0,"""automatic""",null,"""tx""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""abilene""",21995.0,"""ford""","""mustang""","""gas""",33532.0,"""automatic""","""coupe""","""tx""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""abilene""",21950.0,"""gmc""","""sierra 1500""","""gas""",143000.0,"""automatic""",null,"""tx""",2021-04-05 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""akron / canton…",31990.0,"""audi""","""a4 allroad pre…","""other""",22297.0,"""other""","""wagon""","""oh""",2021-04-04 00:00:00 UTC,2021-04-05 00:00:00 UTC
"""abilene""",39500.0,"""ford""","""f-250""","""gas""",139341.0,"""automatic""","""truck""","""tx""",2021-04-06 00:00:00 UTC,2021-04-06 00:00:00 UTC
"""abilene""",27500.0,"""lincoln""","""continental""","""gas""",15912.0,"""automatic""","""sedan""","""tx""",2021-04-05 00:00:00 UTC,2021-04-06 00:00:00 UTC
"""abilene""",23885.0,"""ford""","""f150 xlt 4x4""","""gas""",131169.0,"""automatic""",null,"""tx""",2021-04-05 00:00:00 UTC,2021-04-06 00:00:00 UTC
"""akron / canton…",33590.0,"""lexus""","""gs 350 sedan 4…","""gas""",30814.0,"""automatic""","""sedan""","""oh""",2021-04-06 00:00:00 UTC,2021-04-06 00:00:00 UTC
"""akron / canton…",26995.0,"""ram""","""2500""","""gas""",130960.0,"""automatic""","""other""","""oh""",2021-04-05 00:00:00 UTC,2021-04-06 00:00:00 UTC


## <a id='toc10_2_'></a>[Qualitative evaluation](#toc0_)
Qualitative evaluation of the performance of the ML model.

In [29]:
predicted_sales = hgbr_model.predict(ohe_dataframe(X_test))
print("obs:", len(y_test))
print("total sales (predicted):", sum(predicted_sales))
print("total sales (actual):", sum(y_test))
print("difference:", sum(predicted_sales) - sum(y_test))

The infrequent categories are: 13
obs: 166
total sales (predicted): 165.922655396522
total sales (actual): 167
difference: -1.0773446034779965


# Implement AutoARIMA

In [30]:
def create_sf_data(trainset: pd.DataFrame, groupcols: list) -> pd.DataFrame:
   
   data = (create_removal_date(trainset)
   .groupby(groupcols, as_index=False)
   .aggregate(
      cars_sold = ("price", "count"),
   
   )
   .sort_values("removal_date")
   .reset_index(allow_duplicates=False)
   )

   return data

daily_group_by_cols = ["region", "manufacturer", "model", "fuel", "transmission", "type", "state", "removal_date"]
sf_df = create_sf_data(trainset=X, groupcols=daily_group_by_cols)
sf_df = sf_df.iloc[:-10, :] # we want to predict the last 4 rows
sf_df

,index,region,manufacturer,model,fuel,transmission,type,state,removal_date,cars_sold
0,66,abilene,ford,mustang,gas,automatic,coupe,tx,2021-04-05,1
1,143,akron / canton,audi,a4 allroad premium wagon 4d,other,other,wagon,oh,2021-04-05,1
2,55,abilene,ford,f-250,gas,automatic,truck,tx,2021-04-06,1
3,593,akron / canton,lexus,gs 350 sedan 4d,gas,automatic,sedan,oh,2021-04-06,1
4,87,abilene,lincoln,continental,gas,automatic,sedan,tx,2021-04-06,1
...,...,...,...,...,...,...,...,...,...,...
762,322,akron / canton,dodge,grand caravan,other,automatic,mini-van,oh,2021-06-02,1
763,118,abilene,toyota,tacoma double cab,gas,automatic,pickup,tx,2021-06-03,1
764,65,abilene,ford,flex,gas,automatic,SUV,tx,2021-06-03,1
765,632,akron / canton,mini,clubman,other,automatic,hatchback,oh,2021-06-04,1


In [31]:
def create_sf_train(
    data: pd.DataFrame, 
    encoder: OneHotEncoder = None,
    exclude_cols: list = ["index", "removal_date", "cars_sold"]
    ):
    
    include_cols = [c for c in data.columns if c not in exclude_cols]
    
    # if no encoder is provided, fit one
    if encoder is None:
        encoder = OneHotEncoder(drop="if_binary", max_categories=5, dtype=np.int8, handle_unknown="ignore")
        encoder.fit(data[include_cols])
    
    arr = encoder.transform(data[include_cols]).toarray()
    df = pd.DataFrame(arr, columns=encoder.get_feature_names_out())

    train_df = pd.concat(objs=[df, data[exclude_cols]], axis=1)
    return train_df, encoder

sf_data, sf_ohe = create_sf_train(data=sf_df)
sf_data

,region_akron / canton,manufacturer_chevrolet,manufacturer_ford,manufacturer_jeep,manufacturer_toyota,manufacturer_infrequent_sklearn,model_f-150,model_f-250,model_grand caravan,model_impala,model_infrequent_sklearn,fuel_diesel,fuel_electric,fuel_gas,fuel_other,fuel_infrequent_sklearn,transmission_automatic,transmission_manual,transmission_other,type_SUV,type_other,type_pickup,type_sedan,type_infrequent_sklearn,state_tx,index,removal_date,cars_sold
0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,66,2021-04-05,1
1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,143,2021-04-05,1
2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,55,2021-04-06,1
3,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,593,2021-04-06,1
4,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,87,2021-04-06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,322,2021-06-02,1
763,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,118,2021-06-03,1
764,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,65,2021-06-03,1
765,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,632,2021-06-04,1


In [32]:
# Does each observation need to represent a daily aggregate?
sf = StatsForecast(
    models=[AutoARIMA()],
    freq="W"
)

sf.fit(df=sf_data, id_col="index", time_col="removal_date", target_col="cars_sold")

StatsForecast(models=[AutoARIMA])

In [47]:
# h=1 : predict the same number of rows as the test set

sf.predict(h=1, level=[95, 99], X_df=sf_data.drop(columns=["cars_sold"]))

/Users/johnmwangi/opt/miniconda3/envs/interviews/lib/python3.10/site-packages/statsforecast/arima.py:1607: UserWarning: xreg not required by this model, ignoring the provided regressors
  warnings.warn(
/Users/johnmwangi/opt/miniconda3/envs/interviews/lib/python3.10/site-packages/statsforecast/core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


,removal_date,AutoARIMA,AutoARIMA-lo-99,AutoARIMA-lo-95,AutoARIMA-hi-95,AutoARIMA-hi-99
index,,,,,,
0,2021-05-16,1.0,1.0,1.0,1.0,1.0
1,2021-04-25,1.0,1.0,1.0,1.0,1.0
2,2021-05-09,1.0,1.0,1.0,1.0,1.0
3,2021-05-09,1.0,1.0,1.0,1.0,1.0
4,2021-05-09,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
772,2021-05-09,1.0,1.0,1.0,1.0,1.0
773,2021-05-30,1.0,1.0,1.0,1.0,1.0
774,2021-05-02,1.0,1.0,1.0,1.0,1.0


# <a id='toc11_'></a>[Make predictions](#toc0_)
Prepare data for the dashboard.

In [146]:
predictions = pd.concat(objs=[
    X_test.reset_index(drop=True), 
    pd.Series(y_test, name="actual_sales").reset_index(drop=True),
    pd.Series(predicted_sales, name="predicted_sales"),
    ], axis=1)

predictions

,region,manufacturer,model,transmission,state,price,odometer,year_sold,month_sold,day_sold,season,actual_sales,predicted_sales
0,northern michigan,jeep,wrangler,automatic,mi,-0.004573,0.037201,0,5,9,spring,1,1.008997
1,northern michigan,None,oldsmobile cutlass,automatic,mi,-0.005681,0.046552,0,5,9,spring,1,1.008997
2,norfolk / hampton roads,acura,mdx,automatic,va,-0.005235,0.150470,0,5,9,spring,1,1.008997
3,norfolk / hampton roads,ford,f-250 super duty,automatic,va,-0.006173,-0.312049,0,5,9,spring,1,1.014720
4,norfolk / hampton roads,lincoln,town car,automatic,va,-0.005845,0.415785,0,5,9,spring,1,1.008997
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70666,chicago,nissan,altima 3.5sl,automatic,il,-0.005863,0.249296,0,5,15,spring,1,1.008997
70667,chicago,ford,f-150 lariat,automatic,il,-0.005354,0.179299,0,5,15,spring,1,1.007304
70668,chicago,chevrolet,express 2500,automatic,il,-0.004698,0.140623,0,5,15,spring,1,1.008997
70669,chicago,nissan,altima 2.5,other,il,-0.005499,0.093934,0,5,15,spring,1,1.001651


# <a id='toc12_'></a>[Export data](#toc0_)

In [147]:
def extract_band_values(data: pl.LazyFrame, cols: list):
    res = {}
    for c in cols:
        r = (data
            .unique(subset=f"{c}_band")
            .select(pl.col([f"{c}_band", f"{c}_band_values"]))
            .filter(pl.col(f"{c}_band").is_not_null())
            .collect()
            .sort(by=f"{c}_band")
        )
        
        res[c]=r
        
    return res

In [ ]:
with pd.ExcelWriter(path="./data/predictions.xlsx", mode="w") as writer:
    predictions.to_excel(excel_writer=writer, index=False, sheet_name="predictions")
    # band_values_dict = extract_band_values(band_values, band_cols)
    # for k,v in band_values_dict.items():
    #     v.to_pandas().to_excel(excel_writer=writer, index=False, sheet_name=k)

# <a id='toc13_'></a>[Charts & Graphs](#toc0_)

* You can be able to explore trends, insights, etc of the model over different 
time spans using [this interactive dashboard](https://lookerstudio.google.com/reporting/2803f46f-1fdf-48d0-8bf7-5c6d6a665bd1/page/xEeoD) that has been published.

* The predictions can be found on [this Google Sheets](https://docs.google.com/spreadsheets/d/1gfdVHUMXRjXx1QRTUdIMxKd-FGWmEJnt9DMn1xrgat8/edit#gid=1316071412).


![alt text](./dashboard_ss.png "Screenshot of Craiglist Vehicle Sales Dashboard")